## Using ABM simulation
In ABMs of ecological and evolutionary dynamics, prey naturally grow but get eaten by predators, while the predators grow if they get prey, and naturally die off if they can't find any food.
### Design the data structure to store the attributes of the the prey and predators
The information about about agent type must be represented in the data structure, also in order to simulate the interactions in a spacel, the information about the agents spatial location is also needed. In the code, we use r_init and c_init to represent the initial population of rabbits and coyotes. The for loop iterates r_init + c_init times, and in the first r_init iteration, the prey agents are generated, while the predator agents are generated for the rest.


In [1]:
r_init = 100 # initial rabbit population 
c_init = 30 #initial coyotes population

class agent:
    pass

def initialize():
    global agents
    agents = []
    for i in range(r_init + c_init):
        ag = agent()
        ag.type = 'r' if i < r_init else 'c'
        ag.x = random()
        ag.y = random()
        agents.append(ag)
        

### The rules for how prey and predators behave on their own:
If a prey agent meets a predator agent, it dies with some probability because of predation. We will implement death as the removal of the prey from the preys agents list.
If a predator agent can't find any prey agents nearby, it dies with some probability because of the lack of food. Otherwise, it will reproduce at a certain reproduction rate. 
According to [Purely Facts](http://purelyfacts.com/question/12/which-is-faster-a-coyote-or-a-rabbit?DDA=28&DDB=98),  the top speed is roughly 64 kph for coyote, and 4 kph for rabbit, so let's assume coyotes are 15 times faster than rabbit.

In [13]:
import matplotlib
matplotlib.use('TkAgg')
from pylab import *
import copy as cp

nr = 605 # capacity of rabbits

mr = 0.03 # magnitude of movement of rabbits
dr = 0.8 #death rate of rabbits when it faces coyotes
rr = 0.1 # reproduction rate of rabbits

mc = mr * 15 # magnitude of movement of coyotes
dc = 0.1 # death rate of coyotes when there is no rabbit
rc = 0.5 # reproductioin rate of coyotes

cd = 0.03 # radius for collision detection
cdsq = cd**2

def update():
    global agents
    if agents == []: # Avoiding the possibility for all agents to die out in ecological ABMs
        return
    ag = agents[randint(len(agents))]
    
    #simulating random movement
    m = mr if ag.type == 'r' else mc
    ag.x += uniform(-m, m)
    ag.y += uniform(-m, m)
    ag.x = 1 if ag.x > 1 else 0 if ag.x < 0 else ag.x
    ag.y = 1 if ag.y > 1 else 0 if ag.y < 0 else ag.y
    
    # Detecting collision and simulating death or birth
    neighbors = [nb for nb in agents if (ag.x - nb.x)**2 +(ag.y - nb.y)**2 < cdsq and nb.type != ag.type]
    if ag.type == 'r':
        if len(neighbors ) > 0: #if there are coyotes nearby
            if random() < dr:
                agents.remove(ag)
                return
        if random() < rr * (1 - sum([1 for x in agents if x.type == 'r'])/nr):
            #The logistic-type growth restriction is implemented 
            agents.append(cp.copy(ag))
        
        
    else:
        if len(neighbors) == 0: # If there are no rabbits nearby
            if random() < dc:
                agents.remove(ag)
                return
        else:
            if random() < rc:
                agents.append(cp.copy(ag))
                
def observe():
    global agents
    cla()
    rabbits = [ag for ag in agents if ag.type == 'r']
    if len(rabbits) > 0:
        x = [ag.x for ag in rabbits]
        y = [ag.y for ag in rabbits]
        plot(x,y,'k.')
    coyotes = [ag for ag in agents if ag.type == 'c']
    if len(coyotes) > 0:
        x = [ag.x for ag in coyotes]
        y = [ag.y for ag in coyotes]
        plot(x,y,'ro')
    axis('image')
    axis([0,1,0,1])
                
def update_one_unit_time():
    global agents
    t = 0
    while t < 1:
        t += 1.0 / len(agents)
        update()
import pycxsimulator
pycxsimulator.GUI().start(func = [initialize, observe, update_one_unit_time]) 
